## 1차 테스트 코드

In [4]:
import os
import base64
import json
import csv
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# 인증 설정
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
creds = None

# 토큰 파일이 있다면, 로드하여 인증을 자동으로 처리
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

# 만약 인증 정보가 없다면 새로 인증을 요청
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)

    # 인증 정보를 파일로 저장하여 나중에 사용할 수 있게 함
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

# Gmail API 서비스 객체 생성
service = build('gmail', 'v1', credentials=creds)

# 이메일 메시지 가져오기
def get_messages(service, user_id='me'):
    try:
        # 받은 편지함에서 메일 리스트 가져오기
        results = service.users().messages().list(userId=user_id, labelIds=['INBOX'], q="is:unread").execute()
        messages = results.get('messages', [])

        if not messages:
            print('No messages found.')
            return []
        return messages
    except HttpError as error:
        print(f'An error occurred: {error}')
        return []

# 이메일의 세부 정보 가져오기
def get_message(service, msg_id, user_id='me'):
    try:
        message = service.users().messages().get(userId=user_id, id=msg_id).execute()
        email_data = {}

        # 메일 메시지의 헤더를 가져오기
        if 'payload' in message:
            headers = message['payload'].get('headers', [])
            for header in headers:
                if header['name'] == 'From':
                    email_data['From'] = header['value']
                if header['name'] == 'Subject':
                    email_data['Subject'] = header['value']
            
            # 메시지의 일부 내용 가져오기
            email_data['Snippet'] = message.get('snippet', '')
        
        return email_data
    except HttpError as error:
        print(f'An error occurred: {error}')

# 메일을 CSV로 저장 (UTF-8-SIG 인코딩 사용)
def save_messages_to_csv(service):
    messages = get_messages(service)
    if messages:
        with open('gmail_messages.csv', 'w', newline='', encoding='utf-8-sig') as file:
            writer = csv.DictWriter(file, fieldnames=["From", "Subject", "Snippet"])
            writer.writeheader()

            for message in messages:
                msg = get_message(service, message['id'])
                if msg:  # msg가 None이 아닐 경우만 기록
                    writer.writerow(msg)

# Gmail 데이터 가져오기 및 저장
save_messages_to_csv(service)


In [10]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# CSV 파일 로드
df = pd.read_csv('gmail_messages.csv')

# 데이터 확인
print(df.head())

# 텍스트 정제 함수
def clean_text(text):
    text = text.lower()  # 소문자화
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    text = re.sub(r'[^\w\s]', '', text)  # 특수문자 제거
    return text

# 메일 제목을 정제하여 'cleaned_message' 컬럼에 저장
df['cleaned_message'] = df['Subject'].apply(clean_text)

# TF-IDF 벡터화 (제목 + 본문 내용 모두 사용)
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['cleaned_message'])  # 메일 제목을 TF-IDF로 변환

# K-means 클러스터링 (2개 클러스터로 분류: 정상과 스팸)
kmeans = KMeans(n_clusters=2, random_state=42)
y_kmeans = kmeans.fit_predict(X)

# 클러스터링 결과를 데이터프레임에 추가
df['cluster'] = y_kmeans

# 클러스터 분포 확인
print(df['cluster'].value_counts())

# 클러스터 결과 확인 (정상과 스팸으로 추정된 이메일들)
normal_cluster = df[df['cluster'] == 0]
spam_cluster = df[df['cluster'] == 1]

print("정상 클러스터 예시:")
print(normal_cluster.head())

print("스팸 클러스터 예시:")
print(spam_cluster.head())

# 이제, 클러스터 0을 정상으로, 1을 스팸으로 라벨을 추가할 수 있습니다.
# 클러스터 0을 정상(0), 1을 스팸(1)으로 간주
df['label'] = df['cluster'].map({0: 0, 1: 1})  # 클러스터 0을 정상(0), 1을 스팸(1)으로 간주

# 결과 저장 (UTF-8-SIG 인코딩 사용)
df.to_csv('gmail_messages_with_labels.csv', index=False, encoding='utf-8-sig')

# 라벨을 추가한 데이터 확인
print(df[['Subject', 'label']].head())


                                         From  \
0            Replit <contact@mail.replit.com>   
1  amalgamcaptures <info@amalgamcaptures.com>   
2            Replit <contact@mail.replit.com>   
3   PlayStation <email@email.playstation.com>   
4        Monica Team <noreply@news.monica.im>   

                                             Subject  \
0         Limited time offer + create your dream app   
1  A better Tweed Champ? Meet the Gibson GA-5! Pi...   
2                    💌 Replit으로 만든 나만의 프로젝트를 소개해주세요.   
3                         PlayStation 광고성 정보 수신동의 안내   
4  등록이 성공적으로 완료되었습니다! Monica의 ChatGPT를 이기는 비밀을 발견...   

                                             Snippet  
0  Hi there! As you get comfortable creating apps...  
1  A Champ inspired amp that delivers some beefie...  
2  안녕하세요. Replit 팀입니다. Replit은 누구나 아이디어를 직접 만들고, ...  
3  광고성 정보 수신동의 여부를 안내 드립니다. PlayStation PlayStati...  
4  친애하는 친구, Monica에 오신 것을 환영합니다! ChatGPT의 한계를 넘어서...  
cluster
0    87
1    13
Name: count, d

## 2차 테스트 코드

In [4]:
import os
import base64
import csv
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# 인증 설정
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
creds = None

# 토큰 파일로부터 자격증명 불러오기
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

# 인증이 없거나 유효하지 않으면 새로 인증
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

# Gmail API 서비스 객체 생성
service = build('gmail', 'v1', credentials=creds)


# ✅ 최대 500개까지 이메일 메시지 ID 가져오기
def get_messages(service, user_id='me', max_count=500):
    try:
        messages = []
        next_page_token = None

        while len(messages) < max_count:
            response = service.users().messages().list(
                userId=user_id,
                labelIds=['INBOX'],
                q="is:unread",  # 조건 변경 가능: "after:2024/01/01", "subject:report" 등
                maxResults=min(100, max_count - len(messages)),
                pageToken=next_page_token
            ).execute()

            batch = response.get('messages', [])
            messages.extend(batch)

            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break

        return messages[:max_count]
    except HttpError as error:
        print(f'An error occurred: {error}')
        return []


# ✅ 개별 이메일 메시지 내용 가져오기
def get_message(service, msg_id, user_id='me'):
    try:
        message = service.users().messages().get(userId=user_id, id=msg_id).execute()
        email_data = {}

        headers = message.get('payload', {}).get('headers', [])
        for header in headers:
            if header['name'] == 'From':
                email_data['From'] = header['value']
            elif header['name'] == 'Subject':
                email_data['Subject'] = header['value']

        email_data['Snippet'] = message.get('snippet', '')
        return email_data
    except HttpError as error:
        print(f'An error occurred: {error}')
        return None


# ✅ 수집한 이메일 데이터를 CSV로 저장
def save_messages_to_csv(service, filename='gmail_messages.csv', max_count=500):
    messages = get_messages(service, max_count=max_count)
    if messages:
        with open(filename, 'w', newline='', encoding='utf-8-sig') as file:
            writer = csv.DictWriter(file, fieldnames=["From", "Subject", "Snippet"])
            writer.writeheader()

            for message in messages:
                msg = get_message(service, message['id'])
                if msg:
                    writer.writerow(msg)
        print(f"{len(messages)}건의 메일이 저장되었습니다: {filename}")
    else:
        print("수집된 메일이 없습니다.")


# ✅ 실행
if __name__ == "__main__":
    save_messages_to_csv(service, max_count=500)


500건의 메일이 저장되었습니다: gmail_messages.csv
